<a href="https://colab.research.google.com/github/NehaPunetha/Negation-Unsupervised-STATITICAL/blob/main/Comparision_2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# ==========================
# Common setup
# ==========================
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# -----------------------------
# Load dataset
# -----------------------------
df = pd.read_csv("/content/Amazon_Negation_Reviews_Dataset_500 (2).csv")

print(df['True_Label'].value_counts())
print(df.head())
print(df.columns)

# Column names in this dataset
TEXT_COL  = 'Review'
LABEL_COL = 'True_Label'

# Keep only required columns and drop NaNs
df = df[[TEXT_COL, LABEL_COL]].dropna()

# Encode labels (e.g., Neutral / Positive / Negative → 0,1,2)
label_encoder = LabelEncoder()
df['label_id'] = label_encoder.fit_transform(df[LABEL_COL])

print("\nLabel mapping:")
for cls, idx in zip(label_encoder.classes_, range(len(label_encoder.classes_))):
    print(f"{cls} -> {idx}")

X = df[TEXT_COL].values
y = df['label_id'].values
num_classes = len(label_encoder.classes_)

# Train–test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("\nTrain size:", len(X_train))
print("Test size:", len(X_test))


True_Label
Neutral     237
Positive    132
Negative    131
Name: count, dtype: int64
   ID                                  Review  Has_Negation Negation_Target  \
0   1                      The shoes is slow.             0             NaN   
1   2          I found the shoes quite worst.             0             NaN   
2   3  This shoes turned out to be not worst.             1           worst   
3   4       This phone turned out to be slow.             0             NaN   
4   5                  The pillow is amazing.             0             NaN   

  True_Label  
0   Negative  
1   Negative  
2    Neutral  
3   Negative  
4   Positive  
Index(['ID', 'Review', 'Has_Negation', 'Negation_Target', 'True_Label'], dtype='object')

Label mapping:
Negative -> 0
Neutral -> 1
Positive -> 2

Train size: 400
Test size: 100
True_Label
Neutral     237
Positive    132
Negative    131
Name: count, dtype: int64
   ID                                  Review  Has_Negation Negation_Target  \
0   1   

Shared PyTorch utilities (for DeBERTa & ELECTRA)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

class ReviewsDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = list(texts)
        self.labels = list(labels)
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text  = str(self.texts[idx])
        label = int(self.labels[idx])

        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

def train_epoch(model, data_loader, optimizer, scheduler, device):
    model.train()
    total_loss = 0

    for batch in data_loader:
        optimizer.zero_grad()

        input_ids      = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels         = batch['labels'].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        if scheduler is not None:
            scheduler.step()

    return total_loss / len(data_loader)

def eval_model(model, data_loader, device):
    model.eval()
    preds = []
    true_labels = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids      = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels         = batch['labels'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)

            preds.extend(predictions.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(true_labels, preds)
    cm  = confusion_matrix(true_labels, preds)
    cr  = classification_report(true_labels, preds, target_names=label_encoder.classes_)

    return acc, cm, cr


Using device: cpu


DeBERTa-base (microsoft/deberta-base)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

MODEL_NAME = "microsoft/deberta-base"
max_length = 128
batch_size = 8
epochs = 4
lr = 2e-5

tokenizer_deberta = AutoTokenizer.from_pretrained(MODEL_NAME)

train_dataset_deb = ReviewsDataset(X_train, y_train, tokenizer_deberta, max_length)
test_dataset_deb  = ReviewsDataset(X_test, y_test, tokenizer_deberta, max_length)

train_loader_deb = DataLoader(train_dataset_deb, batch_size=batch_size, shuffle=True)
test_loader_deb  = DataLoader(test_dataset_deb, batch_size=batch_size, shuffle=False)

model_deberta = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=num_classes
).to(device)

optimizer_deb = AdamW(model_deberta.parameters(), lr=lr)
total_steps_deb = len(train_loader_deb) * epochs

scheduler_deb = get_linear_schedule_with_warmup(
    optimizer_deb,
    num_warmup_steps=int(0.1 * total_steps_deb),
    num_training_steps=total_steps_deb
)

for epoch in range(epochs):
    print(f"\n====== DeBERTa Epoch {epoch+1}/{epochs} ======")
    train_loss = train_epoch(model_deberta, train_loader_deb, optimizer_deb, scheduler_deb, device)
    print(f"Train loss: {train_loss:.4f}")

    acc, cm, cr = eval_model(model_deberta, test_loader_deb, device)
    print("Accuracy:", acc)
    print("Confusion Matrix:\n", cm)
    print("Classification Report:\n", cr)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/474 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/559M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/559M [00:00<?, ?B/s]

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



====== DeBERTa Epoch 1/4 ======
Train loss: 0.9063
Accuracy: 0.83
Confusion Matrix:
 [[22  3  1]
 [ 7 35  6]
 [ 0  0 26]]
Classification Report:
               precision    recall  f1-score   support

    Negative       0.76      0.85      0.80        26
     Neutral       0.92      0.73      0.81        48
    Positive       0.79      1.00      0.88        26

    accuracy                           0.83       100
   macro avg       0.82      0.86      0.83       100
weighted avg       0.84      0.83      0.83       100


====== DeBERTa Epoch 2/4 ======
Train loss: 0.3390
Accuracy: 0.85
Confusion Matrix:
 [[26  0  0]
 [ 9 33  6]
 [ 0  0 26]]
Classification Report:
               precision    recall  f1-score   support

    Negative       0.74      1.00      0.85        26
     Neutral       1.00      0.69      0.81        48
    Positive       0.81      1.00      0.90        26

    accuracy                           0.85       100
   macro avg       0.85      0.90      0.85       100

2. ELECTRA-base (google/electra-base-discriminator)

In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

MODEL_NAME = "google/electra-base-discriminator"
max_length = 128
batch_size = 8
epochs = 4
lr = 2e-5

tokenizer_elec = AutoTokenizer.from_pretrained(MODEL_NAME)

train_dataset_elec = ReviewsDataset(X_train, y_train, tokenizer_elec, max_length)
test_dataset_elec  = ReviewsDataset(X_test, y_test, tokenizer_elec, max_length)

train_loader_elec = DataLoader(train_dataset_elec, batch_size=batch_size, shuffle=True)
test_loader_elec  = DataLoader(test_dataset_elec, batch_size=batch_size, shuffle=False)

model_elec = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=num_classes
).to(device)

optimizer_elec = AdamW(model_elec.parameters(), lr=lr)
total_steps_elec = len(train_loader_elec) * epochs

scheduler_elec = get_linear_schedule_with_warmup(
    optimizer_elec,
    num_warmup_steps=int(0.1 * total_steps_elec),
    num_training_steps=total_steps_elec
)

for epoch in range(epochs):
    print(f"\n====== ELECTRA Epoch {epoch+1}/{epochs} ======")
    train_loss = train_epoch(model_elec, train_loader_elec, optimizer_elec, scheduler_elec, device)
    print(f"Train loss: {train_loss:.4f}")

    acc, cm, cr = eval_model(model_elec, test_loader_elec, device)
    print("Accuracy:", acc)
    print("Confusion Matrix:\n", cm)
    print("Classification Report:\n", cr)


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



====== ELECTRA Epoch 1/4 ======

====== ELECTRA Epoch 1/4 ======
Train loss: 1.0115
Train loss: 1.0115
Accuracy: 0.77
Confusion Matrix:
 [[14  8  4]
 [ 2 39  7]
 [ 1  1 24]]
Classification Report:
               precision    recall  f1-score   support

    Negative       0.82      0.54      0.65        26
     Neutral       0.81      0.81      0.81        48
    Positive       0.69      0.92      0.79        26

    accuracy                           0.77       100
   macro avg       0.77      0.76      0.75       100
weighted avg       0.78      0.77      0.76       100


====== ELECTRA Epoch 2/4 ======
Accuracy: 0.77
Confusion Matrix:
 [[14  8  4]
 [ 2 39  7]
 [ 1  1 24]]
Classification Report:
               precision    recall  f1-score   support

    Negative       0.82      0.54      0.65        26
     Neutral       0.81      0.81      0.81        48
    Positive       0.69      0.92      0.79        26

    accuracy                           0.77       100
   macro avg       0

🧱 3. Shared Keras preprocessing (for Self-Attn BiLSTM, TextCNN, BiGRU)

In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_words = 10000
max_len = 50

tokenizer_keras = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer_keras.fit_on_texts(X_train)

X_train_seq = tokenizer_keras.texts_to_sequences(X_train)
X_test_seq  = tokenizer_keras.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post', truncating='post')
X_test_pad  = pad_sequences(X_test_seq, maxlen=max_len, padding='post', truncating='post')

print("X_train_pad shape:", X_train_pad.shape)
print("X_test_pad shape:", X_test_pad.shape)


X_train_pad shape: (400, 50)
X_test_pad shape: (100, 50)
X_train_pad shape: (400, 50)
X_test_pad shape: (100, 50)


In [ ]:
⭐ 3. Self-Attention BiLSTM

In [11]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Embedding, Bidirectional, LSTM, Dense,
    Dropout, Activation, Multiply, Lambda
)
import tensorflow.keras.backend as K

embedding_dim = 128

def self_attention_bilstm_model(max_words, max_len, num_classes):
    inputs = Input(shape=(max_len,))

    x = Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len)(inputs)

    lstm_out = Bidirectional(LSTM(64, return_sequences=True))(x)

    # Self-attention
    score = Dense(1, activation='tanh')(lstm_out)          # (batch, timesteps, 1)
    score = Lambda(lambda z: K.squeeze(z, axis=-1))(score) # (batch, timesteps)
    alphas = Activation('softmax')(score)                  # attention weights

    alphas = Lambda(lambda z: K.expand_dims(z, axis=-1))(alphas)   # (batch, timesteps, 1)
    context = Multiply()([lstm_out, alphas])               # (batch, timesteps, hidden*2)
    context = Lambda(lambda z: K.sum(z, axis=1))(context)  # (batch, hidden*2)

    x = Dropout(0.5)(context)
    x = Dense(64, activation='relu')(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    return model

sa_bilstm_model = self_attention_bilstm_model(max_words, max_len, num_classes)
sa_bilstm_model.summary()

history_sa = sa_bilstm_model.fit(
    X_train_pad,
    y_train,
    epochs=8,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)

y_pred_prob_sa = sa_bilstm_model.predict(X_test_pad)
y_pred_sa = np.argmax(y_pred_prob_sa, axis=1)

print("🔹 Self-Attention BiLSTM")
print("Accuracy:", accuracy_score(y_test, y_pred_sa))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_sa))
print("Classification Report:\n", classification_report(y_test, y_pred_sa, target_names=label_encoder.classes_))


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 50)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 50, 128)   │  1,280,000 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 50, 128)   │     98,816 │ embedding[0][0]   │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 50, 1)     │        129 │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 50)        │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 50)        │          0 │ lambda[0][0]      │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_1 (Lambda)   │ (None, 50, 1)     │          0 │ activation[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 50, 128)   │          0 │ bidirectional[0]… │
│                     │                   │            │ lambda_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_2 (Lambda)   │ (None, 128)       │          0 │ multiply[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 128)       │          0 │ lambda_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 64)        │      8,256 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 3)         │        195 │ dense_1[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,387,396 (5.29 MB)

 Trainable params: 1,387,396 (5.29 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/8
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 151ms/step - accuracy: 0.4664 - loss: 1.0783 - val_accuracy: 0.4250 - val_loss: 1.0865
Epoch 2/8
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 142ms/step - accuracy: 0.4798 - loss: 1.0500 - val_accuracy: 0.4250 - val_loss: 1.0768
Epoch 3/8
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - accuracy: 0.4965 - loss: 1.0443 - val_accuracy: 0.4250 - val_loss: 1.0682
Epoch 4/8
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - accuracy: 0.4536 - loss: 1.0553 - val_accuracy: 0.4250 - val_loss: 1.0475
Epoch 5/8
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - accuracy: 0.4470 - loss: 1.0076 - val_accuracy: 0.5250 - val_loss: 0.8179
Epoch 6/8
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - accuracy: 0.6265 - loss: 0.7105 - val_accuracy: 0.6000 - val_loss: 0.7687
Epoch 7/8
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 82ms/step - accuracy: 0.6157 - loss: 0.6673 - val_accuracy: 0.5250 - val_loss: 0.7698
Epoch 8/8
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - accuracy: 0.6247 - loss: 0.6814 - val_accuracy: 0.5250 - val_los

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


⭐ 4. TextCNN

In [12]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Embedding, Conv1D, GlobalMaxPooling1D,
    Concatenate, Dense, Dropout
)

embedding_dim = 128
filter_sizes = [3, 4, 5]
num_filters = 64

def textcnn_model(max_words, max_len, num_classes):
    inputs = Input(shape=(max_len,))
    x = Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len)(inputs)

    conv_pools = []
    for fsz in filter_sizes:
        conv = Conv1D(filters=num_filters, kernel_size=fsz, activation='relu')(x)
        pool = GlobalMaxPooling1D()(conv)
        conv_pools.append(pool)

    x = Concatenate()(conv_pools)
    x = Dropout(0.5)(x)
    x = Dense(64, activation='relu')(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer='adam',
        metrics=['accuracy']
    )
    return model

textcnn = textcnn_model(max_words, max_len, num_classes)
textcnn.summary()

history_tcnn = textcnn.fit(
    X_train_pad,
    y_train,
    epochs=8,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)

y_pred_prob_tcnn = textcnn.predict(X_test_pad)
y_pred_tcnn = np.argmax(y_pred_prob_tcnn, axis=1)

print("🔹 TextCNN")
print("Accuracy:", accuracy_score(y_test, y_pred_tcnn))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_tcnn))
print("Classification Report:\n", classification_report(y_test, y_pred_tcnn, target_names=label_encoder.classes_))


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 50)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 50, 128)   │  1,280,000 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 48, 64)    │     24,640 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 47, 64)    │     32,832 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 46, 64)    │     41,024 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 64)        │          0 │ conv1d[0][0]      │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 64)        │          0 │ conv1d_1[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 64)        │          0 │ conv1d_2[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 192)       │          0 │ global_max_pooli… │
│ (Concatenate)       │                   │            │ global_max_pooli… │
│                     │                   │            │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 192)       │          0 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 64)        │     12,352 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 3)         │        195 │ dense_3[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,391,043 (5.31 MB)

 Trainable params: 1,391,043 (5.31 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/8
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 69ms/step - accuracy: 0.4376 - loss: 1.0572 - val_accuracy: 0.4250 - val_loss: 0.9771
Epoch 2/8
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.5166 - loss: 0.8993 - val_accuracy: 0.6000 - val_loss: 0.8336
Epoch 3/8
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.6560 - loss: 0.7444 - val_accuracy: 0.8250 - val_loss: 0.7080
Epoch 4/8
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.8309 - loss: 0.5718 - val_accuracy: 0.8000 - val_loss: 0.5866
Epoch 5/8
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 45ms/step - accuracy: 0.8008 - loss: 0.5202 - val_accuracy: 0.8000 - val_loss: 0.5001
Epoch 6/8
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.8655 - loss: 0.4143 - val_accuracy: 0.8500 - val_loss: 0.4406
Epoch 7/8
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.8835 - loss: 0.3341 - val_accuracy: 0.8250 - val_loss: 0.3705
Epoch 8/8
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.8877 - loss: 0.2958 - val_accuracy: 0.8250 - val_loss:

⭐ 5. BiGRU

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, GRU, Dense, Dropout

embedding_dim = 128

bigru_model = Sequential([
    Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_len),
    Bidirectional(GRU(64, return_sequences=False)),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

bigru_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

bigru_model.summary()

history_bigru = bigru_model.fit(
    X_train_pad,
    y_train,
    epochs=8,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)

y_pred_prob_bigru = bigru_model.predict(X_test_pad)
y_pred_bigru = np.argmax(y_pred_prob_bigru, axis=1)

print("🔹 BiGRU")
print("Accuracy:", accuracy_score(y_test, y_pred_bigru))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_bigru))
print("Classification Report:\n", classification_report(y_test, y_pred_bigru, target_names=label_encoder.classes_))


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/8
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 135ms/step - accuracy: 0.4635 - loss: 1.0719 - val_accuracy: 0.4250 - val_loss: 1.0436
Epoch 2/8
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - accuracy: 0.4820 - loss: 1.0003 - val_accuracy: 0.5500 - val_loss: 0.9600
Epoch 3/8
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 84ms/step - accuracy: 0.5863 - loss: 0.8479 - val_accuracy: 0.5250 - val_loss: 0.8448
Epoch 4/8
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - accuracy: 0.6405 - loss: 0.7313 - val_accuracy: 0.6000 - val_loss: 0.7809
Epoch 5/8
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 120ms/step - accuracy: 0.6997 - loss: 0.6178 - val_accuracy: 0.7250 - val_loss: 0.7502
Epoch 6/8
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 82ms/step - accuracy: 0.6974 - loss: 0.5994 - val_accuracy: 0.7750 - val_loss: 0.7412
Epoch 7/8
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - accuracy: 0.6941 - loss: 0.5835 - val_accuracy: 0.6750 - val_loss: 0.7373
Epoch 8/8
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - accuracy: 0.6961 - loss: 0.6004 - val_accuracy: 0.6500 - val_los

1/4 ━━━━━━━━━━━━━━━━━━━━ 1s 428ms/step

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 155ms/step
🔹 BiGRU
Accuracy: 0.71
Confusion Matrix:
 [[19  7  0]
 [ 4 44  0]
 [15  3  8]]
Classification Report:
               precision    recall  f1-score   support

    Negative       0.50      0.73      0.59        26
     Neutral       0.81      0.92      0.86        48
    Positive       1.00      0.31      0.47        26

    accuracy                           0.71       100
   macro avg       0.77      0.65      0.64       100
weighted avg       0.78      0.71      0.69       100

